In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read in data source

df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

#Quick check at first rows
df.head()


In [ ]:
#Check out data
df.describe().T

Min value = 0 in most columns. No bueno.

In [ ]:
#Further Check out data
df.info()

No NULL values. Wooooo!

In [ ]:
#Replace 0's of things that should not equal 0, so we can get a better look at the column values and their associated statistics
df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN)

df.describe()

Min seems to be corrected. All other values were then fixed.  There might be a faster way to replace 0's with NULL for certain columns, just not sure off the top of my head

In [ ]:
df.hist(figsize=(20,20))

I see some outliers and a mostly right skewed distribution.

In [ ]:
#Check to see which columns have null values
df.isnull().sum()

In [ ]:
#first replace NaN values, used median due to the heavy skew right, mean would not be as effective of a fit
df['Glucose'].fillna(df['Glucose'].median(), inplace = True)
df['BloodPressure'].fillna(df['BloodPressure'].median(), inplace = True)
df['SkinThickness'].fillna(df['SkinThickness'].median(), inplace = True)
df['Insulin'].fillna(df['Insulin'].median(), inplace = True)
df['BMI'].fillna(df['BMI'].median(), inplace = True)

In [ ]:
#Recheck to see if there are null values
df.isnull().sum()

In [ ]:
#Check to see if there are any highly correlated inputs
fig, ax = plt.subplots(figsize=(10,10)) 
p=sns.heatmap(df.corr(), annot=True,cmap="BuGn")

Does not seem to be any very highly correlated inputs to the outcome. Glucose has the strongest correlation to the outcome, but it still not very high.

In [ ]:
#Create X and Y datasets
x = df.drop(['Outcome'], axis=1)
y = df['Outcome']

In [ ]:
#Import modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier as KNN

In [ ]:
#Transform data
ssc = StandardScaler()

x_scaled = ssc.fit_transform(x)

In [ ]:
#Split data
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = .3, random_state= 42)

In [ ]:
#Splitting Model and Transforming data

# Instantiate lr
lr = LogisticRegression(random_state=42)

# Instantiate knn
knn = KNN(n_neighbors=10)

# Instantiate dt
dt = DecisionTreeClassifier(min_samples_leaf=.10, random_state=42)


classifiers = [('Logistic Regression', lr), ('K Nearest Neighbours', knn), ('Classification Tree', dt)]


In [ ]:
#Evaluate Each Model
for clf_name, clf in classifiers:    
 
    # Fit clf to the training set
    clf.fit(x_train, y_train)    
   
    # Predict y_pred
    y_pred = clf.predict(x_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred) 
   
    # Evaluate clf's accuracy
    print('{:s} : {:.3f}'.format(clf_name, accuracy))

Classification Tree has highest accuracy

In [ ]:
#Ensemble Model
vc = VotingClassifier(estimators=classifiers)     

#Fit vc to x_train
vc.fit(x_train, y_train)   

#Evaluate the test set predictions
y_pred = vc.predict(x_test)

#Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Voting Classifier: {:.3f}'.format(accuracy))

Ensemble method has second highest accuracy, however Classification Tree is still highest
